In [37]:
!pip install -q cassio datasets langchain openai tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [63]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import  VectorStoreIndexWrapper
from langchain_openai import AzureChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

from datasets import  load_dataset

import cassio

In [39]:
!pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [40]:
from PyPDF2 import PdfReader

In [41]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

In [43]:
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")


In [44]:
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

In [45]:
os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_API_TOKEN")

In [46]:
PDFReader = PdfReader("Projects/Chat With PDF ASTRADB/Linear_Algebra.pdf")

In [47]:
from typing_extensions import Concatenate

raw_text = ""
for i, page in enumerate(PDFReader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

### Initialize connection

In [48]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [77]:
llm = AzureChatOpenAI(model="gpt-35-turbo",api_version="2024-06-01")

In [50]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [51]:
astra_vector_store = Cassandra(
    embedding = embeddings,
    table_name = "QA_demo",
    session=None,
    keyspace=None
)

In [52]:
from langchain.text_splitter import CharacterTextSplitter

In [53]:
from langchain_openai import  AzureOpenAIEmbeddings

In [54]:
embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002")

In [55]:
text_splitter = CharacterTextSplitter(
        separator = "\n",
        chunk_size=800,
        chunk_overlap=200,
        length_function=len,
)


In [56]:
texts = text_splitter.split_text(raw_text)

In [70]:
texts[:10]

['Linear algebra explained in four pages\nExcerpt from the N O BULLSHIT GUIDE TO LINEAR ALGEBRA by Ivan Savov\nAbstract —This document will review the fundamental ideas of linear algebra.\nWe will learn about matrices, matrix operations, linear transformations and\ndiscuss both the theoretical and computational aspects of linear algebra. The\ntools of linear algebra open the gateway to the study of more advanced\nmathematics. A lot of knowledge buzz awaits you if you choose to follow the\npath of understanding , instead of trying to memorize a bunch of formulas.\nI. I NTRODUCTION\nLinear algebra is the math of vectors and matrices. Let nbe a positive\ninteger and let Rdenote the set of real numbers, then Rnis the set of all\nn-tuples of real numbers. A vector ~ v2Rnis ann-tuple of real numbers.',
 'integer and let Rdenote the set of real numbers, then Rnis the set of all\nn-tuples of real numbers. A vector ~ v2Rnis ann-tuple of real numbers.\nThe notation “2S” is read “element of S.” F

In [73]:
astra_vector_store.add_texts(texts[17:20])
print(f"Inserted {len(texts)} headlines")
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 39 headlines


In [ ]:
print(texts[:6])

['Linear algebra explained in four pages\nExcerpt from the N O BULLSHIT GUIDE TO LINEAR ALGEBRA by Ivan Savov\nAbstract —This document will review the fundamental ideas of linear algebra.\nWe will learn about matrices, matrix operations, linear transformations and\ndiscuss both the theoretical and computational aspects of linear algebra. The\ntools of linear algebra open the gateway to the study of more advanced\nmathematics. A lot of knowledge buzz awaits you if you choose to follow the\npath of understanding , instead of trying to memorize a bunch of formulas.\nI. I NTRODUCTION\nLinear algebra is the math of vectors and matrices. Let nbe a positive\ninteger and let Rdenote the set of real numbers, then Rnis the set of all\nn-tuples of real numbers. A vector ~ v2Rnis ann-tuple of real numbers.', 'integer and let Rdenote the set of real numbers, then Rnis the set of all\nn-tuples of real numbers. A vector ~ v2Rnis ann-tuple of real numbers.\nThe notation “2S” is read “element of S.” Fo

In [79]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


QUESTION: "What is Linear Algebra?"


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-06-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}